# 2-Layer Neural Network | Text Colour Predictor

Task:
- Feed-forward neural network.
- Feed RGB values of 'background colour'.
- Predict if dark or light coloured text should be used over the RGB colour to make the text readable.

## Imports

In [ ]:
# Libraries
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Scripts
from rgb import *
from two_layer import *

## Data visualisation

In [ ]:
# Test the RGB class and data visualisation tool functions:
colours = generate_RGB_data(X=1, extreme=True)

for colour in colours:
    colour.generate_img(font_col='#fff')
    plt.imshow(colour.img)
    print('RGB:', colour.RGB, 'Hex:', colour.hex)

## Generate data

In [ ]:
np.random.seed(2) # Optional: set seed for data generation.

colours = generate_RGB_data(X=200, extreme=1)
data = pd.DataFrame([x.RGB for x in colours], columns=['R', 'G', 'B'])

display('Training set:', data)

## Assign Labels with 'lazy' method

In [ ]:
clusterer = AgglomerativeClustering(n_clusters=2, linkage='ward').fit(data.values)
y = clusterer.labels_

In [ ]:
for i, label in enumerate(y[199:]):
    if label == 1:  # NB: must check for most appriate label-to-class assignment.
        print('---> light text')
        colours[i].generate_img(font_col='#fff')
    else:
        print('---> dark text')
        colours[i].generate_img(font_col='#000')
    plt.imshow(colours[i].img)
    plt.show()

## Train/test split

In [ ]:
# Split data into training & testing sets:
train, test = train_test_split(data.join(pd.Series(y, name='y')))

display(train.head(5))
display(test.head(5))

## Use the NN on dummy example

In [ ]:
# Setup inputs:
X = np.array([0, 1]).reshape((2,1))
y = np.array([1, 0]).reshape((2,1))
w1 = np.array([[-1, 0],
               [0, 1]], dtype=float)
w2 = np.array([[1, 0],
               [-1, 1]], dtype=float)

# Initialise NN:
NN = NeuralNetwork(X=X,
                   y=y,
                   bias=1,
                   eta=0.1,
                   w1=w1,
                   w2=w2,
                   num_nodes=2,
                   linear=True)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))

NN.backprop()
print('\nBackpropagation:\nhidden:\n{}\noutput:\n{}'.format(NN.w1, NN.w2))

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))

## Train the NN on training set

In [ ]:
# Setup inputs:
input_X = train.values[0, :3].reshape((3,1))
input_y = train.values[0, 3].reshape((1,1))
print('X:\n{}\ny:\n{}'.format(input_X, input_y))

# Initialise NN:
NN = NeuralNetwork(X=input_X,
                   y=input_y,
                   bias=1,
                   eta=0.75,
                   w1=None,
                   w2=None,
                   num_nodes=10,
                   linear=False)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))

NN.backprop()
print('\nBackpropagation:\nhidden:\n{}\noutput:\n{}'.format(NN.w1, NN.w2))

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))

In [ ]:
# Setup inputs:
tr_i = train.shape[0]
train_Xs = train.values[:, :3].reshape((tr_i, 3, 1))
train_ys = train.values[:, 3].reshape((tr_i, 1))
print('Shapes of inputs:', train_Xs.shape, train_ys.shape)

# Train NN:
train_y_preds = NN.fit(Xs=train_Xs, ys=train_ys, iterations=20)
print('Shape of y_preds:', train_y_preds.shape)

In [ ]:
display(train_y_preds[:5])

In [ ]:
# Training results:
train_results = pd.DataFrame({'y_true': train.y.values,
                              'y_pred': np.round(train_y_preds).reshape((tr_i,)).astype(int),
                              'same': train.y.values == np.round(train_y_preds).reshape((tr_i,)).astype(int)})

display(train_results.head(5))

print('{}% error'.format(round(len(train_results[train_results.same==False]) / len(train_results) * 100)))


In [ ]:
# Setup inputs:
te_i = test.shape[0]
test_Xs = test.values[:, :3].reshape((te_i, 3, 1))
test_ys = test.values[:, 3].reshape((te_i, 1))
print('Shapes of inputs:', test_Xs.shape, test_ys.shape)

# Test NN:
test_y_preds = NN.predict(Xs=test_Xs)
print('Shape of y_preds:', test_y_preds.shape)

In [ ]:
# Testing results:
test_results = pd.DataFrame({'y_true': test.y.values,
                             'y_pred': np.round(test_y_preds).reshape((te_i,)).astype(int),
                             'same': test.y.values == np.round(test_y_preds).reshape((te_i,)).astype(int)})

display(test_results.head(5))

print('{}% error'.format(round(len(test_results[test_results.same==False]) / len(test_results) * 100)))

# NN via scikit-learn

In [ ]:
sklearn_NN = MLPClassifier(activation='logistic')

sklearn_NN.fit(X=train.iloc[:, :3].values, y=train.y.values)

sklearn_y_preds = sklearn_NN.predict(X=test.iloc[:, :3].values)

sklearn_results = pd.DataFrame({'y_true': test.y.values,
                                'y_pred': sklearn_y_preds,
                                'same': test.y.values == sklearn_y_preds})

display(sklearn_results.head(5))

print('{}% error'.format(round(len(sklearn_results[sklearn_results.same==False]) / len(sklearn_results) * 100)))